In [49]:
# keep track of two lists (or a list of pairs) which have number and depth.
# adjacent numbers at the same depth are paired. 
# two numbers at depth 4 are elligible for explode
    # this can be found easily by depth list
    # explode can be carried out by referencing indices

In [3]:
def parse_sfn(sfn):
    values = []
    depths = []
    depth = 0
    for char in sfn:
        if char == '[':
            depth += 1
        elif char == ']':
            depth -= 1
        elif char == ',':
            continue
        else:
            values.append(int(char))
            depths.append(depth)
    return values, depths

In [4]:
def find_explode(depths):
    for ii, pair in enumerate(zip(depths, depths[1:])):
        if pair == (5, 5):
            return ii
    return None

def explode(values, depths, idx):
    # idx is the first of the pair
    left, right = values[idx], values[idx + 1]
#     print(f"{left=}, {right=}")
    # add to left and right if they exist
    if idx > 0:
        values[idx - 1] += left
    if idx < len(values) - 2:
        values[idx + 2] += right
    # replace pair with 0
    
    values[idx] = 0
    del values[idx + 1]
    
    depths[idx] -= 1
    del depths[idx + 1]

In [5]:
def find_split(values):
    for ii, val in enumerate(values):
        if val >= 10:
            return ii
    return None

def split(values, depths, idx):
    to_split = values[idx]
    
    down = round(to_split / 2 - 0.1)
    up = round(to_split / 2 + 0.1)
    
    values[idx] = down
    values.insert(idx + 1, up)
    
    depths[idx] += 1
    depths.insert(idx + 1, depths[idx])

In [15]:
def reduce(values, depths):
    stuff_to_do = True
    while stuff_to_do:
        stuff_to_do = False
        exp_idx = find_explode(depths)
        split_idx = find_split(values)
        if exp_idx is not None:
    #         print("sploding")
            stuff_to_do = True
            explode(values, depths, exp_idx)
        elif split_idx is not None:
    #         print("splitting")
            stuff_to_do = True
            split(values, depths, split_idx)
#     return values, depths

In [34]:
def find_pair(depths):
    for ii, pair in enumerate(zip(depths, depths[1:])):
        if pair[0] == pair[1]:
            return ii
    return None

def magnitude(values, depths):
    values = values.copy()
    depths = depths.copy()
    
    while(len(values) > 1):
        pair_idx = find_pair(depths)
        if pair_idx is not None:
            left_idx, right_idx = pair_idx, pair_idx + 1
            mag = 3 * values[left_idx] + 2 * values[right_idx]
            values[left_idx] = mag
            del values[right_idx]
            depths[left_idx] -= 1
            del depths[right_idx]
    return values[0]

In [22]:
all([
magnitude(*parse_sfn("[[1,2],[[3,4],5]]")) == 143,
magnitude(*parse_sfn("[[[[0,7],4],[[7,8],[6,0]]],[8,1]]")) == 1384,
magnitude(*parse_sfn("[[[[1,1],[2,2]],[3,3]],[4,4]]")) == 445,
magnitude(*parse_sfn("[[[[3,0],[5,3]],[4,4]],[5,5]]")) == 791,
magnitude(*parse_sfn("[[[[5,0],[7,4]],[5,5]],[6,6]]")) == 1137,
magnitude(*parse_sfn("[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]")) == 3488,
])


[143]
[1384]
[445]
[791]
[1137]
[3488]


True

In [17]:
def add_sfn(sfn1, sfn2):
    
    if sfn1 is None:
        return sfn2
    elif sfn2 is None:
        return sfn1
    
    v1, d1 = sfn1
    v2, d2 = sfn2
    
    return v1 + v2, [depth + 1 for depth in d1 + d2]

add_sfn(parse_sfn("[1,2]"), parse_sfn("[[3,4],5]"))

([1, 2, 3, 4, 5], [2, 2, 3, 3, 2])

In [30]:
# fname = 'example.txt'
fname = 'input.txt'
with open(fname) as f:
    lines = f.readlines()
data = [line.strip() for line in lines]

In [35]:
running_sum = None
for sfn in data:
    vals_depths = parse_sfn(sfn)
#     print(f"adding {vals_depths} to {running_sum}")
    running_sum = add_sfn(running_sum, vals_depths)
#     print(f"sum: {running_sum}")
    reduce(*running_sum)
#     print(f"sum: {running_sum}")

magnitude(*running_sum)

3411

In [36]:
max_sum = 0
for ii in range(len(data)):
    for jj in range(len(data)):
            add_result = add_sfn(parse_sfn(data[ii]), parse_sfn(data[jj]))
            reduce(*add_result)
            max_sum = max(magnitude(*add_result), max_sum)
        
print(max_sum)

4680
